In [1]:
!pip install nltk
!pip install networkx
!pip install --upgrade scipy networkx
!pip install transformers
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 45.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
import re
from nltk.corpus import stopwords
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/FYP/Summarization

/content/drive/MyDrive/FYP/Summarization


In [7]:
!python summarize.py --file_location "/content/drive/MyDrive/FYP/Dataset/Class9/Biology/afterCoref/biology_class9_chapter13_coreferenced.txt" --save_location "/content/summary"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['We have seen that cells are the basic units of organisms', ' Cells are made of a variety of chemical substances–proteins, carbo-hydrates, fats or lipids, and so on', ' Cell is a dynamic place', ' Something or the other is always happening inside Cell', ' Complex reactions and repair goes on inside cells', ' New cells are being made', ' In We organs or tissues, there are various specialised activities going on– the heart is beating, the lungs are breathing, the kidney is filtering urine, the brain is thinking', ' All these activities are interconnected', ' For example, if the kidneys are not filtering urine, poisonous substances will accumulate', ' Under such conditions, the brain will not be able to think properly', ' For all these interconnected activities,

In [ ]:
!python summarize.py --folder_location "/content/drive/MyDrive/FYP/Dataset"

2023-03-06 19:28:51.436836: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 19:28:52.909292: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-06 19:28:52.909456: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-06 19:28:52.909482: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [ ]:
def read_document(textfile):
  with open(textfile,"r") as f:
    text_corpus=f.read()
  text_corpus=text_corpus.replace("\n","")
  text_corpus=text_corpus.strip().split(".")
  
  print(text_corpus)
  return text_corpus


In [ ]:
stop_words=stopwords.words("english")

def remove_stopwords(sent):
  new_sent=" ".join([i for i in sent if i not in stop_words])
  return new_sent
  
def preprocess(text):
  sentences=[]
  print(text)
  for t in text:
    sentences.extend(sent_tokenize(t))
  print(sentences)
  cleaned_sentences = pd.Series(sentences)#.str.replace("[^a-zA-Z]", " ", regex=True)
  #cleaned_sentences = [s.lower() for s in cleaned_sentences]
  
  #removing stop words
  #cleaned_sentences = [remove_stopwords(sent.split()) for sent in cleaned_sentences]
  print(cleaned_sentences)

  return cleaned_sentences 

def initialize_gloveembeddings():
  glove_embeddings=dict()

  with open("/content/drive/MyDrive/FYP/Summarization/glove.6B.100d.txt","r") as f:
    for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      glove_embeddings[word] = coefs
    #print(len(glove_embeddings.keys()))
  return glove_embeddings


def get_gloveembeddings(sentences,glove):
  sentence_vectors = []
  for i in sentences:
    if len(i) != 0:
      v = sum([glove.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
      v = np.zeros((100,))
    sentence_vectors.append(v)
  #print(sentence_vectors)
  return sentence_vectors
     


In [ ]:
def generate_summary_textrank(sentences,sentence_vectors,sn):
  sim_matrix = np.zeros([len(sentences), len(sentences)])

  for i in range(len(sentences)):
    for j in range(len(sentences)):
      if i != j:
        sim_matrix[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

  nx_graph = nx.from_numpy_array(sim_matrix)

  scores = nx.pagerank(nx_graph)

  ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
  summary = ""
  # Generate summary
  for i in range(sn):
    summary = summary + ranked_sentences[i][1] + ". "
  #print(summary)
  return summary

In [ ]:
from os import truncate
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
torch_device = 'cuda:0'
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large').cuda()

In [ ]:
def bart_summarize(text):
    num_beams = 4
    #length_penalty = 2.0
    max_length = 800
    #min_length = 100 
    text = text.replace('\n','')
    text_input_ids = tokenizer.batch_encode_plus([text], return_tensors='pt', max_length=1024, truncation=True)['input_ids'].to(torch_device)
    summary_ids = model.generate(text_input_ids, num_beams=int(num_beams), max_length=int(max_length))           
    summary_txt = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
    return summary_txt



In [ ]:
import os
def save_True_statements(folder_location,filename,textrank,bart,combined):
  #print(folder_location,filename)
  summary1=textrank.split(". ")
  summary2=bart.split(". ")
  summary3=combined.split(". ")
  Summarized_text=[]
  Summarized_text.extend(summary1)
  Summarized_text.extend(summary2)
  Summarized_text.extend(summary3)
  Summarized_text=list(set(Summarized_text))
  #print(len(Summarized_text))
  save_folder="Summary"
  save_path=os.path.join(folder_location,save_folder)
  #print(save_path)
  if not os.path.exists(save_path):
    os.mkdir(save_path)
    #print(save_path)
  filename=filename.replace(".txt","-Summarized.txt")
  save_path=os.path.join(save_path,filename)
  print(save_path)
  with open(save_path,"w") as f:
    for sent in Summarized_text:
      if len(sent.split())<=6:
        continue
      f.write(sent)
      f.write("\n")
    print("Saved Successfully at {}".format(save_path))

In [ ]:
from tqdm import tqdm
Datafolder="/content/drive/MyDrive/FYP/Dataset"
Methods=['Textrank','BART','combined']
Metrics=['Rouge-1','Rouge-2','Rouge-L']
Scores=list()
glove=initialize_gloveembeddings()
for dir in tqdm(os.listdir(Datafolder)):
  Classfolder=os.path.join(Datafolder,dir)
  for dir2 in tqdm(os.listdir(Classfolder)):
    subfolder=os.path.join(Classfolder,dir2)
    #try:
    print(subfolder)
    Subject_roguescore=dict()
    Subject_roguescore["Class"]=dir
    Subject_roguescore["Subject"]=dir2
    Subject_roguescore["Method"]=dict()
    for method in Methods:
      Subject_roguescore["Method"][method]=dict()
      for metric in Metrics:
        Subject_roguescore["Method"][method][metric]=list()
    for file in os.listdir(os.path.join(subfolder,"afterCoref")):
      
      filename=file
      file_location=os.path.join(subfolder,"afterCoref",file)
      folder_location=subfolder
      print(file_location)
      corpus=read_document(file_location)
      #print(len(corpus))
      cleaned=preprocess(corpus)
      
      sentence_vectors=get_gloveembeddings(cleaned,glove)
      #Textrank Algorithm
      Num_sent_generate=min(len(cleaned),30)
      summary=generate_summary_textrank(cleaned,sentence_vectors,Num_sent_generate).strip(" ")
      from rouge import Rouge 
      print("\n***TextRank Summary***\n")
      #print(summary)
      scoring = Rouge()
      
      
      print("\n***ROUGE Score***\n")
      Rogue_score=scoring.get_scores(summary, corpus)
      print(Rogue_score)
      Subject_roguescore["Method"]["Textrank"]["Rouge-1"].append(Rogue_score[0]['rouge-1']['f'])
      Subject_roguescore["Method"]["Textrank"]["Rouge-2"].append(Rogue_score[0]['rouge-2']['f'])
      Subject_roguescore["Method"]["Textrank"]["Rouge-L"].append(Rogue_score[0]['rouge-l']['f'])
     
      #Bart Summarization
      bs = bart_summarize(corpus)
      print("\n***BART Summary***\n")
      #print(bs)
      scoring = Rouge()
      print("\n***ROUGE Score***\n")
      Rogue_score=scoring.get_scores(bs, corpus)
      print(Rogue_score)
      Subject_roguescore["Method"]["BART"]["Rouge-1"].append(Rogue_score[0]['rouge-1']['f'])
      Subject_roguescore["Method"]["BART"]["Rouge-2"].append(Rogue_score[0]['rouge-2']['f'])
      Subject_roguescore["Method"]["BART"]["Rouge-L"].append(Rogue_score[0]['rouge-l']['f'])
      #print(scores)
      combined = bs + ". " +summary
      combinedsum = bart_summarize(combined)
      print("\n***Combined Summary***\n")
      #print(combinedsum)
      scoring = Rouge()
      print("\n***ROUGE Score***\n")
      Rogue_score=scoring.get_scores(combined, corpus)
      print(Rogue_score)
      Subject_roguescore["Method"]["combined"]["Rouge-1"].append(Rogue_score[0]['rouge-1']['f'])
      Subject_roguescore["Method"]["combined"]["Rouge-2"].append(Rogue_score[0]['rouge-2']['f'])
      Subject_roguescore["Method"]["combined"]["Rouge-L"].append(Rogue_score[0]['rouge-l']['f'])
      save_True_statements(folder_location,filename,summary,bs,combinedsum)
    #except:
    #  continue
    for method in Methods:
      for metric in Metrics:
        Subject_roguescore["Method"][method][metric]=sum(Subject_roguescore["Method"][method][metric])/len(Subject_roguescore["Method"][method][metric])
    
    Scores.append(Subject_roguescore)





  0%|          | 0/2 [00:00<?, ?it/s]

/content/drive/MyDrive/FYP/Dataset/Class9/Biology
/content/drive/MyDrive/FYP/Dataset/Class9/Biology/afterCoref/biology_class9_chapter5_coreferenced.txt
Cells were first discovered by Robert Hooke in 1665. Robert Hooke observed Cells in a cork slice with the help of a primitive microscope. Leeuwenhoek, with the improved microscope, discovered the free living cells in pond water for the first time. It was Robert Brown in 1831 who discovered the nucleus in the cell. Purkinje in 1839 coined the term ‘protoplasm’ for the fluid substance of the cell. The cell theory, that all the plants and animals are composed of cells and that the cell is the basic unit of life, was presented by two biologists, Schleiden and Schwann. The cell theory, that all the plants and animals are composed of cells and that the cell is the basic unit of life was further expanded by Virchow by suggesting that all cells arise from pre-existing cells. With the discovery of the electron microscope in 1940, it was possible


 50%|█████     | 1/2 [01:57<01:57, 117.43s/it]

[{'rouge-1': {'r': 1.0, 'p': 0.8809523809523809, 'f': 0.9367088557795226}, 'rouge-2': {'r': 0.972, 'p': 0.6952789699570815, 'f': 0.810675558106874}, 'rouge-l': {'r': 1.0, 'p': 0.8809523809523809, 'f': 0.9367088557795226}}]
/content/drive/MyDrive/FYP/Dataset/Class9/Biology biology_class9_chapter7_coreferenced.txt
62
/content/drive/MyDrive/FYP/Dataset/Class9/Biology/Summary
/content/drive/MyDrive/FYP/Dataset/Class9/Biology/Summary/biology_class9_chapter7_coreferenced-Summarized.txt
Saved Successfully at /content/drive/MyDrive/FYP/Dataset/Class9/Biology/Summary/biology_class9_chapter7_coreferenced-Summarized.txt
/content/drive/MyDrive/FYP/Dataset/Class9/Geography
/content/drive/MyDrive/FYP/Dataset/Class9/Geography/afterCoref/geography_class9_chapter5_coreferenced.txt
With about 47,000 plant species India occupies tenth place in the world and fourth in Asia in plant diversity. There are about 15,000 flowering plants in India, which account for 6 per cent in the world’s total number of flow


 50%|█████     | 1/2 [06:20<06:20, 380.23s/it]

...

/content/drive/MyDrive/FYP/Dataset/Class8/Biology
/content/drive/MyDrive/FYP/Dataset/Class8/Biology/afterCoref/biology_class8_chapter1_coreferenced.txt
Animals including humans can not make Animals including humans own food. energy from food is utilised by organisms for carrying out organisms various body functions, such as digestion, respiration and excretion. humans receive food from plants, or animals, or both.In order to provide food for a large population— regular production, proper management and distribution is necessary.Till 10,000 B.C.E. people were nomadic. 10,000 B.C.E. people were nomadic were wandering in groups from place to place in search of food and shelter. 10,000 B.C.E. people were nomadic ate raw fruits and vegetables and started hunting animals for food. Later, 10,000 B.C.E. people were nomadic could cultivate land and produce rice, wheat and other food crops. Thus, was born ‘Agriculture’.When plants of the same kind are cultivated at one place on a large scale, ‘A


 50%|█████     | 1/2 [01:59<01:59, 119.64s/it]

[{'rouge-1': {'r': 0.931924882629108, 'p': 0.9232558139534883, 'f': 0.9275700884580531}, 'rouge-2': {'r': 0.8854415274463007, 'p': 0.8135964912280702, 'f': 0.8479999950089405}, 'rouge-l': {'r': 0.9295774647887324, 'p': 0.9209302325581395, 'f': 0.9252336398599224}}]
/content/drive/MyDrive/FYP/Dataset/Class8/Biology biology_class8_chapter7_coreferenced.txt
77
/content/drive/MyDrive/FYP/Dataset/Class8/Biology/Summary
/content/drive/MyDrive/FYP/Dataset/Class8/Biology/Summary/biology_class8_chapter7_coreferenced-Summarized.txt
Saved Successfully at /content/drive/MyDrive/FYP/Dataset/Class8/Biology/Summary/biology_class8_chapter7_coreferenced-Summarized.txt
/content/drive/MyDrive/FYP/Dataset/Class8/Geography
/content/drive/MyDrive/FYP/Dataset/Class8/Geography/afterCoref/geography_class8_chapter4_coreferenced.txt
Primary activities include all those connected with extraction and production of natural resources. Agriculture, fishing and gathering are good examples. Secondary activities are con


100%|██████████| 2/2 [10:24<00:00, 312.15s/it]

[{'rouge-1': {'r': 0.7982142857142858, 'p': 0.9390756302521008, 'f': 0.8629343579672337}, 'rouge-2': {'r': 0.7063829787234043, 'p': 0.8137254901960784, 'f': 0.7562642319269824}, 'rouge-l': {'r': 0.7982142857142858, 'p': 0.9390756302521008, 'f': 0.8629343579672337}}]
/content/drive/MyDrive/FYP/Dataset/Class8/Geography geography_class8_chapter6_coreferenced.txt
87
/content/drive/MyDrive/FYP/Dataset/Class8/Geography/Summary
/content/drive/MyDrive/FYP/Dataset/Class8/Geography/Summary/geography_class8_chapter6_coreferenced-Summarized.txt
Saved Successfully at /content/drive/MyDrive/FYP/Dataset/Class8/Geography/Summary/geography_class8_chapter6_coreferenced-Summarized.txt


In [ ]:
print(Scores)

 

[{'Class': 'Class9', 'Subject': 'Biology', 'Method': {'Textrank': {'Rouge-1': 0.8413227880185278, 'Rouge-2': 0.7722291840674762, 'Rouge-L': 0.8413227880185278}, 'BART': {'Rouge-1': 0.7114577896075013, 'Rouge-2': 0.566690089995877, 'Rouge-L': 0.7059969582165141}, 'combined': {'Rouge-1': 0.8394230692927062, 'Rouge-2': 0.7180577688690605, 'Rouge-L': 0.838431743394317}}}, {'Class': 'Class9', 'Subject': 'Geography', 'Method': {'Textrank': {'Rouge-1': 0.6303335196452368, 'Rouge-2': 0.5490615576105738, 'Rouge-L': 0.6303335196452368}, 'BART': {'Rouge-1': 0.5563361772550693, 'Rouge-2': 0.441891358167179, 'Rouge-L': 0.5539483230705229}, 'combined': {'Rouge-1': 0.7355842507122382, 'Rouge-2': 0.637022591637542, 'Rouge-L': 0.7344786728716572}}}, {'Class': 'Class8', 'Subject': 'Biology', 'Method': {'Textrank': {'Rouge-1': 0.752784273129885, 'Rouge-2': 0.6660984620409526, 'Rouge-L': 0.752784273129885}, 'BART': {'Rouge-1': 0.6001955377560395, 'Rouge-2': 0.46696547509696407, 'Rouge-L': 0.59927852721041

{'a': '1'}
{'a': '1', 'b': '2'}


In [ ]:
import json
with open("./Summarizer_metrics.json","w") as f:
  json.dump(Scores,f)

In [ ]:
#Bart Summarization
bs = bart_summarize(corpus)

print("\n***BART Summary***\n")
print(bs)
scoring = Rouge()
print("\n***ROUGE Score***\n")
print(scoring.get_scores(bs, corpus))


***BART Summary***

Climate refers to the sum total of weather conditions and variations over a large area for a long period of time. The elements of weather and climate are the same, i.e. temperature, atmospheric pressure, wind, humidity and precipitation. There are six major controls of the climate of any place. they are: latitude, altitude, pressure and wind system, distance from the sea (continentality), ocean currents and relief features. Due to the curvature of the earth, the amount of solar energy received varies according to latitude. As a result, air temperature generally decreases from the equator towards the poles. In Asia, this type of climate is found mainly in the south and the southeast. High mountains act as barriers for cold or hot winds; they may also cause precipitation if they are high enough and lie in the path of rain-bearing winds. The leeward side of mountains remains relatively dry. The hills are therefore cooler during summers. As one goes from the surface to

In [ ]:
combined = bs + ". " +summary
combinedsum = bart_summarize(combined)

print("\n***Combined Summary***\n")
print(combinedsum)
scoring = Rouge()
print("\n***ROUGE Score***\n")
print(scoring.get_scores(combinedsum, corpus))


***Combined Summary***

Climate refers to the sum total of weather conditions and variations over a large area for a long period of time. The elements of weather and climate are the same, i.e. temperature, atmospheric pressure, wind, humidity and precipitation. All these conditions are governed by the elevation of the place. There are six major controls of the climate of any place. they are: latitude, altitude, pressure and wind system, distance from the sea (continentality), ocean currents and relief features. Let us take two important elements – temperature and precipitation, and examine how they vary from place to place and season to season. Due to the curvature of the earth, the amount of solar energy received varies according to latitude. As a result, air temperature generally decreases from the equator towards the poles. In general, coastal areas experience less contrasts in temperature conditions. As one goes from the surface to higher altitudes, the atmosphere becomes less den

In [ ]:
corpus=read_document("/content/chapter5Output.txt")
cleaned=preprocess(corpus)
glove=initialize_gloveembeddings()
sentence_vectors=get_gloveembeddings(cleaned,glove)

#Textrank Algorithm
summary=generate_summary_textrank(cleaned,sentence_vectors,30).strip(" ")

from rouge import Rouge 
print("\n***TextRank Summary***\n")
print(summary)
scoring = Rouge()
print("\n***ROUGE Score***\n")
print(scoring.get_scores(summary, test_corpus))
#Bart Summarization
bs = bart_summarize(test_corpus)

print("\n***BART Summary***\n")
#print(bs)
scoring = Rouge()
print("\n***ROUGE Score***\n")
print(scoring.get_scores(bs, test_corpus))
#print(scores)
combined = bs + ". " +summary
combinedsum = bart_summarize(combined)
print("\n***Combined Summary***\n")
#print(combinedsum)
scoring = Rouge()
print("\n***ROUGE Score***\n")
print(scoring.get_scores(combinedsum, test_corpus))


['Our planet, Earth is the only one on which life, as we know it, exists', 'Most life-forms we know need an ambient temperature, water, and food', 'These comprise the hydrosphere', 'Living things are found where these three exist', 'Living things constitute the biotic component of the biosphere', 'The Breath of Life: Air We have already talked about the composition of air in the first chapter', 'It is interesting to note that even the composition of air is the result of life on Earth', 'In fact, carbon dioxide constitutes up to 95-97% of the atmosphere on Venus and Mars', 'This results in the production of carbon dioxide', 'This includes not just human activities, which burn fuels to get energy, but also forest fires', 'We have talked of the atmosphere covering the Earth, like a blanket', 'The atmosphere keeps the average temperature of the Earth fairly steady during the day and even during the course of the whole year', 'And during the night, it slows down the escape of heat into oute

In [ ]:
a=summary.split(". ")
for i in a:
  print(i)
  print("\n")

The type of soil is decided by the average size of particles found in it and the quality of the soil is decided by the amount of humus and the microscopic organisms found in it


However, some of the factors that created the soil in the first place and brought the soil to that place may be responsible for the removal of the soil too


Removal of useful components from the soil and addition of other substances, which adversely affect the fertility of the soil and kill the diversity of organisms that live in it, is called soil pollution


Another manner in which the temperature of the water in rivers can be affected is when water is released from dams


This condensation of water is facilitated if some particles could act as the nucleus for these drops to form around


Other than these bacteria, the only other manner in which the nitrogen molecule is converted to nitrates and nitrites is by a physical process


The atmosphere can be heated from below by the radiation that is reflected ba

In [ ]:
a=dict()
a["As"]=dict(dict(list()))
a["As"]["a"].append(a)
print(a)

KeyError: ignored

In [ ]:
{"C":{"meth1":{"metric1":[],"metric2":[]},"meth2":{"metric1":[],"metric2":[]}}}

{'C': {'meth1': {'metric1': [], 'metric2': []},
  'meth2': {'metric1': [], 'metric2': []}}}